In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/My Drive/UITws-v1'

/content/gdrive/My Drive/UITws-v1


In [3]:
import sklearn.svm as svm
import multiprocessing as mp
from ws_utils import WSUtils
from ws_features import WSCountVectorizer

import time
import datetime

In [4]:
start = time.time()

print(datetime.datetime.now())

2020-07-26 08:13:29.123539


In [5]:
cpu_count = mp.cpu_count() # Number of CPUs for parallel
print(cpu_count)

4


In [6]:
w_utils               = WSUtils('RDR_VNDict') # Utils object for Vietnamese Word Segmentation
predict_funct         = w_utils.predict_list_of_sentence_ws # Function for prediction after training
training_sentences    = w_utils.read_ws_corpus('./data/VLSP2013_WS_train_gold.txt') # Read sentences for training

X_syls_train, Y_train = w_utils.extract_training_pairs(training_sentences) # Extract training window of syllables
ratios                = w_utils.compute_ratios(training_sentences) # Extract separable syllables
seen_words, seen_sfx  = w_utils.pop_seen_words_sfx(training_sentences, ratios) # Extract know words and unknow words containing suffixes

vectorizer_WS         = WSCountVectorizer(utils=w_utils, ratios=ratios, feature_set=['base', 'long', 'sep', 'sfx'])
X_train               = vectorizer_WS.fit_transform(X_syls_train) # Transformation function

In [7]:
model_WS              = svm.LinearSVC(C=0.1) # Linear SVM model
model_WS.fit(X_train, Y_train) # Train Vietnamese Word segmentation model
ws                    = {'model': model_WS, 'vectorizer': vectorizer_WS}

In [8]:
test_sentences        = w_utils.read_ws_corpus('./data/VLSP2013_WS_test_gold.txt') # Read sentences for testing

pred_stns_test, prf_t = predict_funct(ws, cpu_count, test_sentences, False, has_underscore=True) # Predict

print([round(100*i, 2) for i in prf_t]) # Precision, Recall, F1-score

[97.81, 98.57, 98.19]


In [9]:
end = time.time()
print(end - start)

print(datetime.datetime.now())

239.30247330665588
2020-07-26 08:17:28.426146
